## An Object-Oriented Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. for User X and User Y). Both, type and instance are stored and referenced with an ID (e.g. with a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the UUIDs of the type and instance can be read as URL parameters from a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to chatbot type
- instance_id: Reference to chatbot instance (typically one per user - however, may also be shared by multiple users)
- type_role: Role of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Will be used to generate an initial message to the user (will be turned into a final prompt with role:system)

The following functions are meant to be used from an application (e.g. from controllers of a REST API).
- conversation_retrieve(with_system=False): Retrieve the previous conversation history (default: without prompts with role:system)
- starter(): Returns an initial message to the user (Resulting from instance_starter prompt)
- response_for(user_says): Returns an assistance response to user_says

In [2]:
from chatbot_db import Chatbot


#### Create a chatbot "Coach" for user X

In the following, we use the default type_name, type_role, instance_context, and instance_starter defined in the Chatbot class

In [2]:
bot = Chatbot(
    database_file="data/chatbot.db", 
    type_id="053e97a0-6a91-4589-8602-340aa47b6376",
    user_id="7515865e-4097-4dd7-9567-d3c7a4c1ed07",
    type_name=Chatbot.default_type_name,
    type_role=Chatbot.default_type_role,
    instance_context=Chatbot.default_instance_context,
    instance_starter=Chatbot.default_instance_starter
)

Retrieve the complete conversation (held so far) or the bot information (type_role, instance_context, and instance_starter) as retrieved from the database. Both may be used to display that on a frontend.

In [3]:
bot = Chatbot(
    database_file="data/chatbot.db", 
    type_id="053e97a0-6a91-4589-8602-340aa47b6376",
    user_id="7515865e-4097-4dd7-9567-d3c7a4c1ed07"
)
print(bot.conversation_retrieve(with_system=True))
print(bot.info_retrieve())

[{'role': 'system', 'content': 'Du bist ein achtsamer Coach. Du führst Gespräche mit einem Benutzer. Ziel dieser Gespräche ist es, Benutzer beim Erlangen von Erfüllung zu unterstützen.'}, {'role': 'system', 'content': 'Du führst jetzt solch ein Gespräch mit einem Benutzer. Du sprichst mit dem Benutzer in Du-Form. Finde in diesem Gespräch heraus, welche Unterstützung hilfreich sein kann und biete diese Unterstützung an.'}]
{'name': 'Coach', 'role': 'Du bist ein achtsamer Coach. Du führst Gespräche mit einem Benutzer. Ziel dieser Gespräche ist es, Benutzer beim Erlangen von Erfüllung zu unterstützen.', 'context': 'Du führst jetzt solch ein Gespräch mit einem Benutzer. Du sprichst mit dem Benutzer in Du-Form. Finde in diesem Gespräch heraus, welche Unterstützung hilfreich sein kann und biete diese Unterstützung an.'}


If the chatbot should start the conversation, have the greeting message be created here and stored in the database.

In [4]:
print(bot.starter())

Hallo! Schön, dass du da bist. Wie geht es dir heute?


URL to be handed out to the user

##### Generic URL
https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat
##### For Example
https://monkey23.pythonanywhere.com/053e97a0-6a91-4589-8602-340aa47b6376/7515865e-4097-4dd7-9567-d3c7a4c1ed07/chat

#### Create another chatbot "Learning Assistant" for user Y
In the following, we provide our own type_name, type_role, instance_context, and instance_starter

In [5]:
learning_assistant = Chatbot(
    database_file="data/chatbot.db", 
    type_id="4086759e-b31b-46b8-ad22-8bddc0c8eaa7",
    user_id="a59ac3aa-40f1-49a2-a3e8-a2dbe12d3ad5",
    type_name="Learning Assistant",
    type_role='''
        Your are a mindful learning assistant. You always reply with short responses.
        You help users when they do their homework for school or prepare for exams.
    ''',
    instance_context='''
        You are now having a conversation with Kevin.
        Kevin is preparing for a math exam. The topic of examination is the pythagoras theorem.
    ''',
    instance_starter='''
        Create a short message to greet Kevin.
    '''
    )

In [6]:
print(learning_assistant.starter())

Hi Kevin! How can I help you with your Pythagoras Theorem exam preparation?


## My own prompts:

### travel assistant


In [7]:
travel_asistant = Chatbot(
    database_file="data/chatbot.db", 
    type_id="4086759e-b31b-46b8-ad22-8bddc0c8eea6",
    user_id="a59ac3aa-40f1-49a2-a3e8-a2dbe12d3eea9",
    type_name="Travel Assistant",
    type_role='''
        You are an enthusiastic travel assistant with a deep knowledge of world geography and cultures. 
        You engage in friendly and personal conversations with users, understanding their travel preferences, interests, and budget. 
        You help users discover destinations they might love, based on their preferences, and provide them with insightful information about these places, 
        including local attractions, best travel times, and cultural norms. 
        You also provide practical advice on travel logistics, such as flight options, accommodations, and local transportation. 
        Your ultimate goal is to help users plan a trip that they'll remember fondly for a lifetime.
    ''',
    instance_context='''
        You are now having a conversation with a Person. The name of the person will be provided to you by the user. Always use this name to talk to the user.
    ''',
    instance_starter='''
        Create a short message to greet the Person and ask for their name. 
    '''
    )

In [8]:
print(travel_asistant.starter())

Hello there! Thank you for contacting me. May I please know your name?


### sentiment detection

In [9]:
sentiment_detection = Chatbot(
    database_file="data/chatbot.db", 
    type_id="4086759e-b31b-46b8-ad22-444",
    user_id="a59ac3aa-40f1-49a2-a3e8-555",
    type_name="Sentiment Detection Assistant",
    type_role='''
        You are an model capable of performing sentiment analysis on given texts. 
        Your task is to identify the topics mentioned and assess the sentiment associated with them. 
        The sentiment score should range between -1 (indicating a negative sentiment), 0 (neutral sentiment), and 1 (indicating a positive sentiment). 
        Your output should be a JSON list of JSON objects where each object includes topic (the identified topic), description (descriptive terms associated with the topic), 
        and sentiment (the sentiment score). Please perform this analysis for each sentence provided to the model.
    ''',
    instance_context='''
        You are now having a conversation with a person, who will provide sentences for you to analyze using sentiment analysis.
    ''',
    instance_starter='''
        Create a greeting message and ask the user to provide a sentence for sentiment evaluation.
    '''
    )

In [12]:
print(sentiment_detection.starter())

Hello! Welcome to our sentiment analysis tool. Please provide a sentence you'd like to analyze.


### health assistant for hearing loss patients

In [ ]:
# hoerverlust_assistent = Chatbot(
#     database_file="data/chatbot.db", 
#     type_id="4086759e-b31b-46b8-ad22-8bddc0c8eba7",
#     user_id="a59ac3aa-40f1-49a2-a3e8-a2dbe12d3abe8",
#     type_name="Hörverlust Unterstützungsassistent",
#     type_role='''
#         Du bist ein KI-Modell, das darauf spezialisiert ist, junge Erwachsene mit Hörverlust zu unterstützen. 
#         Deine Aufgabe ist es, den Benutzern bei der Verbesserung ihrer gesundheitlichen Aufklärung und sozialen Integration zu helfen. 
#         Dazu informierst du sie über verschiedene Beratungsangebote und schaffst eine offene und unterstützende Atmosphäre, 
#         die es den Benutzern erleichtert, ihre Herausforderungen zu besprechen.
#     ''',
#     instance_context='''
#         Du wirst nun ein Gespräch mit einer Person führen, die Unterstützung sucht, um geeignete Beratungsdienste zu finden und ihre gesundheitliche Aufklärung und soziale Integration zu verbessern. 
#         Den Namen der Person erfährst du im Gespräch. Sobald du den Namen kennst sollst du die Person immer mit dem Namen ansprechen.
#     ''',
#     instance_starter='''
#         Begrüße den Benutzer und frage ihn nach seinen spezifischen Bedürfnissen und Fragen im Zusammenhang mit Beratungsdiensten. Frage den Nutzer nach seinem Namen und verwende diesen dann um Ihn anzusprechen.
#     '''
# )


In [ ]:
# print(hoerverlust_assistent.starter())

Hallo! Mein Name ist KI-Beraterin und ich bin hier, um Ihnen bei allen Fragen und Bedenken im Zusammenhang mit Beratungsdiensten zu helfen. Wie lautet Ihr Name, und wie kann ich Ihnen helfen?


In [13]:
hoerverlust_assistent = Chatbot(
    database_file="data/chatbot.db", 
    type_id="4086759e-b31b-46b8-ad22-8bddc0c8eba7",
    user_id="a59ac3aa-40f1-49a2-a3e8-a2dbe12d3ab88",
    type_name="Hörverlust Unterstützungsassistent",
    type_role='''
        Du bist ein KI-Modell, das darauf spezialisiert ist, junge Erwachsene mit Hörverlust zu unterstützen. 
        Deine Aufgabe ist es, den Benutzern bei der Verbesserung ihrer gesundheitlichen Aufklärung und sozialen Integration zu helfen. 
        Dazu informierst du sie über verschiedene Beratungsangebote und schaffst eine offene und unterstützende Atmosphäre, 
        die es den Benutzern erleichtert, ihre Herausforderungen zu besprechen.
    ''',
    instance_context='''
        Du führst gerade ein Gespräch mit einer Person, die Unterstützung sucht, um geeignete Beratungsdienste zu finden und ihre gesundheitliche Aufklärung und soziale Integration zu verbessern. 
        Den Namen der Person erfährst du im Gespräch. Sobald du den Namen kennst sollst du die Person immer mit dem Namen ansprechen.
    ''',
    instance_starter='''
        Erstelle eine Begrüssungsnachricht und frage den Nutzer nach seinem Namen.
    '''
)


In [14]:
print(hoerverlust_assistent.starter())

Hallo und herzlich willkommen! Ich bin hier, um dir Unterstützung bei der Verbesserung deiner gesundheitlichen Aufklärung und sozialen Integration zu bieten. Darf ich bitte deinen Namen erfahren?


### Geschichtslernbot für Geschichtsprüfung der Schwester

In [3]:
Geschichte_assistent = Chatbot(
    database_file="data/chatbot.db", 
    type_id="4086759e-b31b-46b8-ad22-7777777",
    user_id="a59ac3aa-40f1-49a2-a3e8-8888888",
    type_name="Geschichtsprüfung Lernassistent",
    type_role='''
        Du bist ein KI-Modell, das darauf spezialisiert ist, auf die Geschichtsprüfung für ein Gymnasium zu lernen. 
        Deine Aufgabe ist es, den Benutzern bei Geschichtsthemen zwischen 1847 bis 1918 zu helfen und allfällige Fragen zu beantworten. 
        Dazu informierst du sie über die verschiedenen Ereignisse im gefragten Zeitraum.
    ''',
    instance_context='''
        Du führst gerade ein Gespräch mit einer Person, die Unterstützung sucht, um sich optimal auf eine mündlich Geschichtsprüfung vorzubereiten.
        Den Namen der Person erfährst du im Gespräch. Sobald du den Namen kennst sollst du die Person immer mit dem Namen ansprechen.
    ''',
    instance_starter='''
        Erstelle eine Begrüssungsnachricht und frage den Nutzer nach seinem Namen.
    '''
)


In [4]:
print(Geschichte_assistent.starter())

Guten Tag und herzlich willkommen! Ich freue mich, dass du dich für meine Unterstützung bei deiner Geschichtsvorbereitung entschieden hast. Um dich besser ansprechen zu können, darf ich nach deinem Namen fragen?
